"Sensor Suara Berbasis Machine Learning"

Anggota Kelompok:

1. Alysha Putri Anggadinata

2. Fajris Zahrotun Nihayah

3. Shereva Miranda

In [ ]:
!pip install paho-mqtt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for paho-mqtt: filename=paho_mqtt-1.6.1-py3-none-any.whl size=62118 sha256=4c12674647507d2aa086845c6f0262d952a6f81a586710c9b08315992aca30af
  Stored in directory: /root/.cache/pip/wheels/8b/bb/0c/79444d1dee20324d442856979b5b519b48828b0bd3d05df84a
Successfully built paho-mqtt


In [ ]:
import numpy as np
import paho.mqtt.client as mqtt
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# MQTT broker information
mqtt_broker = "test.mosquitto.org"
mqtt_port = 1883
mqtt_topic = "reciever-elka"  # Set the MQTT topic to match the one used in Node-RED
mqtt_result_topic = "nodered-elka-result"  # Set the MQTT topic to publish the classification result

# Constants for sound classification
num_classes = 3
feature_size = 1  # Modify this based on the number of features collected from Arduino

# Variables to store parsed data
features = []
labels = []

def on_connect(client, userdata, flags, rc):
    print("Connected to MQTT broker")
    client.subscribe(mqtt_topic)

classification_results = []
class_names = ['Quiet', 'Moderate', 'High']

def on_message(client, userdata, msg):
    message = msg.payload.decode()  # Decode the received message

    # Parse the received message and extract features and label
    # Modify this part based on the structure and format of your data
    # Assuming the received message is a comma-separated list of features followed by the label
    data = message.split(',')

    # Extract features
    feature_values = data[:feature_size]
    features.append(feature_values)

    # Extract label
    label = int(data[feature_size])
    labels.append(label)

    # Perform classification and publish the result
    if features >= 100:
        # Convert the parsed data into numpy arrays
        X = np.array(features, dtype=np.float32)
        y = np.array(labels, dtype=np.uint8)

        # Preprocess the data (e.g., normalize feature values)
        X_normalized = X / np.max(X)

        # Reshape the data to match CNN input shape (num_samples, height, width, channels)
        X_normalized = X_normalized.reshape(-1, feature_size, 1, 1)

        # Convert labels to one-hot encoded vectors
        y_encoded = keras.utils.to_categorical(y, num_classes=num_classes)

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X_normalized, y_encoded, test_size=0.2, random_state=42)

        # Build the CNN model
        model = keras.Sequential([
            layers.Conv2D(16, kernel_size=(3, 1), activation="relu", input_shape=(feature_size, 1, 1)),
            layers.MaxPooling2D(pool_size=(2, 1)),
            layers.Flatten(),
            layers.Dense(64, activation="relu"),
            layers.Dense(num_classes, activation="softmax")
        ])

        # Compile the model
        model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

        # Train the model
        batch_size = 16
        epochs = 10

        # Training loop
        for epoch in range(epochs):
            history = model.fit(X_train, y_train, batch_size=batch_size, epochs=1, validation_data=(X_test, y_test))

        # Perform classification
        y_pred = model.predict(X_normalized)
        y_pred_labels = np.argmax(y_pred, axis=1)

        # Publish the classification result
        for pred_label in y_pred_labels:
            client.publish(mqtt_result_topic, str(pred_label))
            print("Classification Result:", class_names[pred_label])

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message